Access the embeddings from kaggle API mentioned in the notrbooks

## Import dependencies

In [2]:
import pandas as pd
import os
import tensorflow as tf
from tensorboard.plugins import projector

2024-05-29 21:51:02.617015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Import data

In [3]:
df = pd.read_csv("infersent_embeddings.csv",nrows=30000,index_col=0) # change the path according to your local
embedding_tensor = tf.convert_to_tensor(df.drop(['prompt'],axis=1))

In [6]:
prompts = df['prompt'].str.replace('\n','').astype(str)

## Plotting using tensorboard

In [7]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='logs/embedding/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for prompt in prompts:
    f.write("{}\n".format(prompt))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(embedding_tensor)
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

run ``` tensorboard --logdir logs/embedding``` and use the available umap,t-sne,pca vizualizations